In [1]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer

from xgboost import plot_importance
import matplotlib.pyplot as plt

from utils.query_snowflake import SnowflakeConnector

import mlflow

C:\Users\User\PycharmProjects\pums\venv\lib\site-packages\snowflake\connector\options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
person_df = pd.read_csv( r"C:\Users\User\Downloads\pums\psam_p06.csv")
housing_df = pd.read_csv( r"C:\Users\User\Downloads\pums\psam_h06.csv")

In [3]:
housing_df = housing_df[[col for col in housing_df.columns if col not in ['RT', 'PUMA10', 'PUMA20', 'DIVISION', 'REGION', 'ST', 'ADJINC']]]

In [4]:
df = person_df.merge(housing_df, on="SERIALNO", how="left")

In [5]:
df = df[~df['FS'].isna()]
df = df.reset_index(drop=True)
df["FS"] = df["FS"].map({1: 1, 2: 0})

In [6]:
# 1. Define groups
income_fields = ["FINCP", "HINCP"] + ["INTP", "OIP", "PAP", "PERNP", "PINCP", "RETP", "SEMP", "SSIP", "SSP", "WAGP"]
housing_fields = ["ELEP", "CONP", "FULP", "GASP", "INSP", "MHP", "MRGP", "RNTP", "SMP", 
                  "WATP", "GRNTP", "SMOCP", "TAXAMT", "CONP",]

# 2. Adjust to 2022 dollars
for field in income_fields:
    df[field] = df[field] * df["ADJINC"] / 1_000_000

for field in housing_fields:
    df[field] = df[field] * df["ADJHSG"] / 1_000_000

In [7]:
allowed_codes = [
    '11','21','22','23','31','32','33','3M','42','44','45','48','49',
    '4M','51','52','53','54','55','56','61','62','71','72','81','92','99','na'
]

df['NAICSP_clean'] = df['NAICSP'].fillna('na').apply(
    lambda x: str(x)[:2] if str(x)[:2] in allowed_codes else 'na'
)

naicsp_dummies = pd.get_dummies(
    pd.Categorical(df['NAICSP_clean'], categories=allowed_codes),
    prefix='NAICSP'
)

# Step 3: Combine and drop original columns
df = pd.concat([df.drop(columns=['NAICSP', 'NAICSP_clean']), naicsp_dummies], axis=1)

In [8]:
allowed_socp_codes = [
    '11','13','15','17','19','21','23','25','27','29','31','33',
    '35','37','39','41','43','45','47','49','51','53','55','99','na'
]

# Step 1: Clean SOCP values to 2-digit prefix or 'na'
df['SOCP_clean'] = df['SOCP'].fillna('na').apply(
    lambda x: str(x)[:2] if str(x)[:2] in allowed_socp_codes else 'na'
)

# Step 2: One-hot encode with fixed categories
socp_dummies = pd.get_dummies(
    pd.Categorical(df['SOCP_clean'], categories=allowed_socp_codes),
    prefix='SOCP'
)

# Step 3: Drop original columns and combine
df = pd.concat([df.drop(columns=['SOCP', 'SOCP_clean']), socp_dummies], axis=1)

df = df.astype({col: 'float64' for col in df.select_dtypes(include='int').columns})

In [9]:
# Derived ratios
df["income_per_person"] = df["HINCP"] / (df["NP"].replace(0, np.nan))
df["income_per_room"] = df["HINCP"] / (df["RMSP"].replace(0, np.nan))
df["rent_to_income_ratio"] = df["RNTP"] / (df["HINCP"].replace(0, np.nan))

C:\Users\User\AppData\Local\Temp\ipykernel_9732\1510629749.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["income_per_person"] = df["HINCP"] / (df["NP"].replace(0, np.nan))
C:\Users\User\AppData\Local\Temp\ipykernel_9732\1510629749.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["income_per_room"] = df["HINCP"] / (df["RMSP"].replace(0, np.nan))
C:\Users\User\AppData\Local\Temp\ipykernel_9732\1510629749.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

In [10]:
import json
RUN_ID = '5f4a1acc4e674bf19d7280f239446f5d'
train_config_path = f"C:/Users/User/PycharmProjects/pums/notebooks/mlruns/635971186787891896/{RUN_ID}/artifacts/train.config"
with open(train_config_path) as f:
    config = json.load(f)

In [11]:
import mlflow
logged_model = f'runs:/{RUN_ID}/model'
loaded_model = mlflow.sklearn.load_model(logged_model)

client = mlflow.tracking.MlflowClient()
run = client.get_run(RUN_ID)
params = run.data.params
custom_threshold =  float(params.get("custom_threshold"))
print(custom_threshold)

0.3505


In [12]:
# Raw prediciton (default threshold = 0.5)
predictions = loaded_model.predict(pd.DataFrame(df[config['features']])).astype(int)

# Predict probabilities for the positive class (FS = 1)
proba = loaded_model.predict_proba(pd.DataFrame(df[config['features']]))[:, 1]
predictions_custom_threshold = (proba >= custom_threshold).astype(int)

In [13]:
pred_df = df[['RT', 'SERIALNO']].reset_index(drop=True)
pred_df['FS_pred'] = predictions.tolist()
pred_df['FS_pred_custom_threshold'] = predictions_custom_threshold.tolist()
pred_df['FS_true'] = df['FS'].tolist()

In [14]:
from sklearn.metrics import classification_report

print(classification_report(
    pred_df['FS_true'],
    pred_df['FS_pred'],
    sample_weight=df['PWGTP']
))

              precision    recall  f1-score   support

         0.0       0.86      0.99      0.92 33389182.0
         1.0       0.68      0.11      0.20 5848654.0

    accuracy                           0.86 39237836.0
   macro avg       0.77      0.55      0.56 39237836.0
weighted avg       0.84      0.86      0.81 39237836.0



In [15]:
from sklearn.metrics import classification_report

print(classification_report(
    pred_df['FS_true'],
    pred_df['FS_pred_custom_threshold'],
    sample_weight=df['PWGTP']  
))

              precision    recall  f1-score   support

         0.0       0.87      0.98      0.92 33389182.0
         1.0       0.60      0.19      0.29 5848654.0

    accuracy                           0.86 39237836.0
   macro avg       0.74      0.58      0.60 39237836.0
weighted avg       0.83      0.86      0.83 39237836.0



In [18]:
len(pred_df.SERIALNO.unique()), len(pred_df)

(153864, 386061)

In [22]:
df

,RT,SERIALNO,DIVISION,SPORDER,PUMA_x,REGION,ST,ADJINC,PWGTP,AGEP,...,SOCP_47,SOCP_49,SOCP_51,SOCP_53,SOCP_55,SOCP_99,SOCP_na,income_per_person,income_per_room,rent_to_income_ratio
0,P,2021GQ0000005,9.0,1.0,3741.0,4.0,6.0,1029928.0,177.0,26.0,...,False,False,False,False,False,False,True,NaN,NaN,NaN
1,P,2021GQ0000006,9.0,1.0,11102.0,4.0,6.0,1029928.0,14.0,80.0,...,False,False,False,False,False,False,True,NaN,NaN,NaN
2,P,2021GQ0000007,9.0,1.0,6712.0,4.0,6.0,1029928.0,14.0,47.0,...,False,False,False,False,False,False,True,NaN,NaN,NaN
3,P,2021GQ0000021,9.0,1.0,5500.0,4.0,6.0,1029928.0,19.0,61.0,...,False,False,False,False,False,False,False,NaN,NaN,NaN
4,P,2021GQ0000057,9.0,1.0,3735.0,4.0,6.0,1029928.0,17.0,26.0,...,False,False,False,False,False,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386056,P,2021HU1415655,9.0,1.0,11300.0,4.0,6.0,1029928.0,70.0,73.0,...,False,False,False,False,False,False,True,17261.59328,14384.661067,NaN
386057,P,2021HU1415655,9.0,2.0,11300.0,4.0,6.0,1029928.0,52.0,73.0,...,False,False,False,False,False,False,True,17261.59328,14384.661067,NaN
386058,P,2021HU1415655,9.0,3.0,11300.0,4.0,6.0,1029928.0,63.0,42.0,...,False,False,False,False,False,False,False,17261.59328,14384.661067,NaN
386059,P,2021HU1415655,9.0,4.0,11300.0,4.0,6.0,1029928.0,62.0,36.0,...,False,False,False,False,False,False,False,17261.59328,14384.661067,NaN
